In [23]:
import re
import logging
import os
import fitz  # PyMuPDF
import pdfplumber
from unidecode import unidecode
import chardet
from collections import Counter
import hashlib
import tempfile
# A biblioteca 'docling' é uma dependência externa.
# Certifique-se de que ela está instalada: pip install docling
from docling.document_converter import DocumentConverter

# Configuração avançada de logging para registrar saídas em arquivo e no console.
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("functional_pdf_processor.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# --- Funções de Avaliação e Extração de Texto ---

def evaluate_text_quality(text: str) -> float:
    """Avalia a qualidade do texto extraído com base em métricas como caracteres válidos e espaços."""
    text_len = len(text)
    if text_len < 100:
        return 0.0

    valid_chars = sum(1 for c in text if c.isalnum() or c.isspace() or c in ',.;:!?()[]{}@#$%&*_+-=/')
    valid_ratio = valid_chars / text_len

    space_ratio = text.count(' ') / text_len
    space_score = 0.1 if 0.1 < space_ratio < 0.3 else 0

    problem_chars = sum(text.count(char) for char in ['�', '\x00', '\ufffd'])
    problem_ratio = problem_chars / text_len
    problem_penalty = 1 - min(problem_ratio * 10, 1)

    score = (valid_ratio * 0.7) + (space_score * 0.1) + (problem_penalty * 0.2)
    return score

def extract_with_fitz(file_path: str) -> str:
    """Extrai texto usando PyMuPDF (fitz)."""
    try:
        with fitz.open(file_path) as doc:
            return "".join(page.get_text("text", sort=True) + "\n" for page in doc)
    except Exception as e:
        logger.error(f"Falha na extração com PyMuPDF: {e}")
        return ""

def extract_with_pdfplumber(file_path: str) -> str:
    """Extrai texto usando pdfplumber."""
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
            return text
    except Exception as e:
        logger.error(f"Falha na extração com pdfplumber: {e}")
        return ""

def extract_best_text(file_path: str) -> str:
    """Tenta extrair texto com múltiplos métodos e escolhe o melhor resultado."""
    methods = {"PyMuPDF": extract_with_fitz, "pdfplumber": extract_with_pdfplumber}
    best_text = ""
    best_score = -1.0
    
    for name, method_func in methods.items():
        logger.info(f"Tentando extração com {name}...")
        text = method_func(file_path)
        if not text:
            logger.warning(f"{name} não retornou texto.")
            continue
        
        score = evaluate_text_quality(text)
        logger.info(f"Qualidade do texto extraído com {name}: {score:.2f}")

        if score > best_score:
            best_text = text
            best_score = score
            
    if not best_text:
        logger.error("Todas as técnicas de extração de texto falharam.")
    
    return best_text

# --- Funções de Limpeza, Correção e Formatação ---

def detect_language(text: str) -> str:
    """Detecta o idioma principal do texto (português ou inglês)."""
    sample_text = text[:5000].lower()
    pt_indicators = sum(sample_text.count(word) for word in [' de ', ' a ', ' o ', ' que ', ' e '])
    en_indicators = sum(sample_text.count(word) for word in [' the ', ' and ', ' of ', ' to ', ' in '])
    
    if pt_indicators > en_indicators * 1.2:
        return "portuguese"
    elif en_indicators > pt_indicators * 1.2:
        return "english"
    return "unknown"

def analyze_errors(text: str) -> tuple[dict, Counter]:
    """Analisa o texto em busca de erros de codificação e retorna um mapa de correções."""
    error_patterns = Counter()
    correction_map = {}
    
    common_fixes = {
        'Ã§': 'ç', 'Ã£': 'ã', 'Ã¡': 'á', 'Ã©': 'é', 'Ã³': 'ó', 'Ãµ': 'õ', 
        'Ãª': 'ê', 'Ã¢': 'â', 'Ã ': 'à', 'Ãº': 'ú', 'Ã­': 'í', 'PîS': 'Pós',
        'Gradua,ÌO': 'Graduação', 'CAPêTULO': 'CAPÍTULO', 'â€œ': '"', 
        'â€': '"', 'â€™': "'", 'â€“': '-'
    }
    
    for error, fix in common_fixes.items():
        if error in text:
            count = text.count(error)
            error_patterns[error] += count
            correction_map[error] = fix

    hyphenated = re.findall(r'\b\w+-\s*\n\s*\w+\b', text)
    error_patterns["hyphenated_word"] += len(hyphenated)
    
    logger.info(f"Padrões de erro mais comuns detectados: {error_patterns.most_common(5)}")
    return correction_map, error_patterns

def auto_correct(text: str, correction_map: dict, language: str) -> str:
    """Aplica as correções identificadas e outras limpezas gerais."""
    for error, correction in correction_map.items():
        text = text.replace(error, correction)
    
    text = re.sub(r'(\b\w+)-\s*\n\s*(\w+\b)', r'\1\2', text, flags=re.IGNORECASE)
    text = re.sub(r'[ \t]+', ' ', text)
    
    if language == "portuguese":
        text = re.sub(r'([a-zà-ÿ])\.\s+([A-ZÀ-Ÿ])', r'\1.\n\n\2', text)
        
    return text

def enhance_structure(text: str) -> str:
    """Converte elementos estruturais do texto em sintaxe Markdown."""
    text = re.sub(r'\n\s*(CAP[ÍI]TULO|CHAPTER)\s+([IVXLCDM\d]+)[\s.-]*\n', r'\n## \1 \2\n', text, flags=re.IGNORECASE)
    text = re.sub(r'\n\s*(\d+(\.\d+)+)[\s.-]+([^\n]+)', r'\n### \1 \3', text)
    text = re.sub(r'\n\s*([-*•])\s+', r'\n- ', text)
    return text

def sanitize_text(text: str) -> str:
    """Remove caracteres inválidos e de controle."""
    sanitized = re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)
    replacements = {'\ufffd': '', '\xad': '', '\u200b': ''}
    for char, replacement in replacements.items():
        sanitized = sanitized.replace(char, replacement)
    sanitized = re.sub(r'\n{3,}', '\n\n', sanitized)
    return sanitized.strip()

def process_with_docling(text: str) -> str:
    """Processa o texto com a biblioteca Docling para formatação final."""
    tmp_path = ""
    try:
        sanitized_text = sanitize_text(text)
        with tempfile.NamedTemporaryFile(mode='w', suffix='.md', delete=False, encoding='utf-8') as tmp:
            tmp.write(sanitized_text)
            tmp_path = tmp.name
        
        logger.info("Processando com a biblioteca Docling...")
        converter = DocumentConverter()
        result = converter.convert(tmp_path)
        return result.document.export_to_markdown()
    except Exception as e:
        logger.error(f"Falha no processamento com Docling: {e}. Retornando texto pré-processado.")
        return text
    finally:
        if tmp_path and os.path.exists(tmp_path):
            os.unlink(tmp_path)

# --- Orquestrador Principal ---

def process_pdf_file(input_path: str, output_dir: str):
    """
    Executa todo o pipeline de processamento para um único arquivo PDF.
    """
    logger.info(f"Iniciando processamento de: {os.path.basename(input_path)}")
    if not os.path.exists(input_path):
        logger.error(f"Arquivo não encontrado: {input_path}")
        return

    # 1. Extração
    raw_text = extract_best_text(input_path)
    if not raw_text:
        logger.error(f"Não foi possível extrair texto de {input_path}. Abortando.")
        return
    logger.info(f"Texto extraído com sucesso ({len(raw_text)} caracteres).")

    # 2. Análise
    language = detect_language(raw_text)
    logger.info(f"Idioma detectado: {language}")
    correction_map, _ = analyze_errors(raw_text)

    # 3. Correção e Formatação
    corrected_text = auto_correct(raw_text, correction_map, language)
    structured_text = enhance_structure(corrected_text)
    
    # 4. Processamento Final e Saída
    final_text = process_with_docling(structured_text)

    base_name = os.path.splitext(os.path.basename(input_path))[0]
    output_path = os.path.join(output_dir, f"{base_name}.md")
    
    try:
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(final_text)
        logger.info(f"Documento processado e salvo em: {output_path}")
        
        text_hash = hashlib.md5(final_text.encode()).hexdigest()
        logger.info(f"MD5 do conteúdo final: {text_hash}")
    except IOError as e:
        logger.error(f"Falha ao salvar o arquivo de saída {output_path}: {e}")

def main():
    """Função principal para configurar e executar o processamento."""
    PDF_FILES = ['Regulamento_Aprovado_2019.pdf'] # Coloque seus arquivos aqui
    OUTPUT_DIR = "processed_documents_functional"
    
    os.makedirs(OUTPUT_DIR, exist_ok=True)
    
    for pdf_file in PDF_FILES:
        if not os.path.isfile(pdf_file):
            logger.error(f"Arquivo '{pdf_file}' não encontrado. Pulando.")
            continue
        
        process_pdf_file(pdf_file, OUTPUT_DIR)
        logger.info(f"Processamento de {pdf_file} concluído.\n" + "="*50)

if __name__ == "__main__":
    main()


INFO:__main__:Iniciando processamento de: Regulamento_Aprovado_2019.pdf
INFO:__main__:Tentando extração com PyMuPDF...
INFO:__main__:Qualidade do texto extraído com PyMuPDF: 0.90
INFO:__main__:Tentando extração com pdfplumber...
INFO:__main__:Qualidade do texto extraído com pdfplumber: 0.91
INFO:__main__:Texto extraído com sucesso (37981 caracteres).
INFO:__main__:Idioma detectado: portuguese
INFO:__main__:Padrões de erro mais comuns detectados: [('hyphenated_word', 2)]
INFO:__main__:Processando com a biblioteca Docling...
INFO:docling.document_converter:Going to convert document batch...
INFO:docling.document_converter:Initializing pipeline for SimplePipeline with options hash 4cc01982ae99b46a2a63fcda46c47c35
INFO:docling.pipeline.base_pipeline:Processing document tmpo2qiiivw.md
INFO:docling.document_converter:Finished converting document tmpo2qiiivw.md in 0.22 sec.
INFO:__main__:Documento processado e salvo em: processed_documents_functional/Regulamento_Aprovado_2019.md
INFO:__main__

In [4]:
import re
import logging
import os
import fitz  # PyMuPDF
import pdfplumber
import requests
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# ==============================================================================
# 1. CONFIGURAÇÕES GERAIS
# ==============================================================================
# --- Arquivos de Entrada e Saída ---
PDF_PATHS = ['Regulamento_Aprovado_2019.pdf']  # Coloque seus PDFs aqui
MARKDOWN_OUTPUT_DIR = "documentos_processados"

# --- Configuração do RAG ---
USER_QUESTION = "Quais são os regulamentos para o dainf?"
MODEL_EMBEDDING = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
OLLAMA_API_URL = "http://10.20.50.50:11434/api/generate"
MODEL_LLM = "deepseek-r1:14b"
THRESHOLD = 0.25
MAX_CONTEXT_SIZE = 120000

# ==============================================================================
# 2. CONFIGURAÇÃO DE LOGGING
# ==============================================================================
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("rag_system.log", mode='w'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

# ==============================================================================
# 3. FUNÇÕES DE PROCESSAMENTO DE PDF (OTIMIZADAS)
# ==============================================================================

def evaluate_text_quality(text: str) -> float:
    if len(text) < 100: 
        return 0.0
    valid_chars = sum(1 for c in text if c.isalnum() or c in ' ,.;:!?()[]{}@#$%&*_+-=/')
    return valid_chars / len(text)

def extract_with_fitz(file_path: str) -> str:
    try:
        with fitz.open(file_path) as doc:
            return "\n".join(page.get_text("text", sort=True) for page in doc)
    except Exception as e:
        logger.error(f"[PDF] Falha no PyMuPDF: {str(e)}")
        return ""

def extract_with_pdfplumber(file_path: str) -> str:
    try:
        with pdfplumber.open(file_path) as pdf:
            return "\n".join(page.extract_text() for page in pdf.pages if page.extract_text())
    except Exception as e:
        logger.error(f"[PDF] Falha no pdfplumber: {str(e)}")
        return ""

def extract_best_text(file_path: str) -> str:
    methods = [
        ("PyMuPDF", extract_with_fitz),
        ("pdfplumber", extract_with_pdfplumber)
    ]
    
    best_text, best_score = "", -1
    for name, method in methods:
        text = method(file_path)
        if not text:
            continue
            
        score = evaluate_text_quality(text)
        logger.info(f"[PDF] {name} - Score: {score:.2f}")
        if score > best_score:
            best_text, best_score = text, score
    
    return best_text if best_score > 0.3 else ""

def enhance_structure(text: str) -> str:
    # Padronização de cabeçalhos
    text = re.sub(r'\n\s*(CAP[ÍI]TULO|SEÇÃO|ARTIGO)\s+([IVXLCDM\d]+)[\s.-]*\n', 
                 r'\n## \1 \2\n', text, flags=re.IGNORECASE)
    
    # Identificação de subseções
    text = re.sub(r'\n\s*(\d+[\.\d]*)\s+([^\n]+)', r'\n### \1 \2', text)
    
    # Listas
    text = re.sub(r'\n\s*([•▪‣➢])\s*', r'\n- ', text)
    
    # Espaçamento entre parágrafos
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    return text.strip()

def process_pdf_to_markdown(file_path: str, output_dir: str) -> str:
    base_name = os.path.splitext(os.path.basename(file_path))[0]
    output_path = os.path.join(output_dir, f"{base_name}.md")
    
    logger.info(f"[PDF] Processando: {os.path.basename(file_path)}")
    raw_text = extract_best_text(file_path)
    
    if not raw_text:
        logger.error(f"[PDF] Falha na extração de texto: {file_path}")
        return ""
    
    structured_text = enhance_structure(raw_text)
    
    try:
        os.makedirs(output_dir, exist_ok=True)
        with open(output_path, "w", encoding="utf-8") as f:
            f.write(structured_text)
        logger.info(f"[PDF] Documento salvo: {output_path}")
        return output_path
    except Exception as e:
        logger.error(f"[PDF] Erro ao salvar Markdown: {str(e)}")
        return ""

# ==============================================================================
# 4. SISTEMA RAG OTIMIZADO
# ==============================================================================

def load_documents():
    """Carrega todos os documentos processados em memória"""
    documents = []
    document_paths = []
    
    for pdf_path in PDF_PATHS:
        if not os.path.exists(pdf_path):
            logger.error(f"Arquivo não encontrado: {pdf_path}")
            continue
            
        md_path = process_pdf_to_markdown(pdf_path, MARKDOWN_OUTPUT_DIR)
        if not md_path or not os.path.exists(md_path):
            continue
            
        try:
            with open(md_path, "r", encoding="utf-8") as f:
                content = f.read()
                documents.append(content)
                document_paths.append(md_path)
                logger.info(f"Documento carregado: {os.path.basename(md_path)}")
        except Exception as e:
            logger.error(f"Erro ao ler documento: {str(e)}")
    
    return documents, document_paths

def get_embeddings(documents: list[str]):
    """Gera embeddings para todos os documentos"""
    try:
        logger.info(f"Carregando modelo: {MODEL_EMBEDDING}")
        model = SentenceTransformer(MODEL_EMBEDDING)
        return model.encode(documents)
    except Exception as e:
        logger.error(f"Falha no embedding: {str(e)}")
        raise

def rag_query(question: str, documents: list[str], document_paths: list[str], embeddings):
    """Executa todo o pipeline RAG"""
    try:
        # 1. Embedding da pergunta
        embedding_model = SentenceTransformer(MODEL_EMBEDDING)
        question_embedding = embedding_model.encode([question])[0]
        
        # 2. Cálculo de similaridade
        similarities = cosine_similarity([question_embedding], embeddings)[0]
        
        # 3. Seleção de contexto
        context = ""
        logger.info("\nSimilaridade dos documentos:")
        for i, (path, sim) in enumerate(zip(document_paths, similarities)):
            status = "✅" if sim > THRESHOLD else "❌"
            logger.info(f"{status} {os.path.basename(path)}: {sim:.4f}")
            
            if sim > THRESHOLD:
                context += f"\n--- DOCUMENTO: {os.path.basename(path)} ---\n"
                context += documents[i] + "\n\n"
        
        # 4. Truncagem segura
        if len(context) > MAX_CONTEXT_SIZE:
            logger.warning(f"Contexto truncado ({len(context)} > {MAX_CONTEXT_SIZE} chars)")
            context = context[:MAX_CONTEXT_SIZE]
        
        # 5. Construção do prompt
        prompt = f"""<|im_start|>system
Você é um assistente acadêmico que responde perguntas baseado EXCLUSIVAMENTE nos documentos fornecidos. 
Se a informação não estiver contida nos documentos, responda claramente que não possui os dados necessários.<|im_end|>
<|im_start|>context
{context}<|im_end|>
<|im_start|>user
Pergunta: {question}<|im_end|>
<|im_start|>assistant
Resposta:"""
        
        # 6. Chamada à API
        payload = {
            "model": MODEL_LLM,
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.3,
                "top_p": 0.9,
                "num_ctx": 4096
            }
        }
        
        logger.info(f"Consultando LLM: {MODEL_LLM}")
        response = requests.post(
            OLLAMA_API_URL,
            json=payload,
            headers={"Content-Type": "application/json"},
            timeout=180
        )
        response.raise_for_status()
        
        # 7. Processamento da resposta
        raw_response = response.json().get('response', '')
        answer = raw_response.split("<|im_end|>")[0].replace("Resposta:", "").strip()
        
        # 8. Exibição do resultado
        print("\n" + "="*80)
        print(f"PERGUNTA: {question}")
        print("="*80)
        print(answer)
        print("="*80)
        
    except Exception as e:
        logger.error(f"Erro no RAG: {str(e)}")

# ==============================================================================
# 5. EXECUÇÃO PRINCIPAL
# ==============================================================================

if __name__ == "__main__":
    # Etapa 1: Processar e carregar documentos
    documents, document_paths = load_documents()
    if not documents:
        logger.error("Nenhum documento válido carregado. Abortando.")
        exit(1)
    
    # Etapa 2: Gerar embeddings
    try:
        embeddings = get_embeddings(documents)
    except Exception:
        exit(1)
    
    # Etapa 3: Executar consulta RAG
    rag_query(USER_QUESTION, documents, document_paths, embeddings)

2025-06-20 14:14:43,275 - INFO - [PDF] Processando: Regulamento_Aprovado_2019.pdf
2025-06-20 14:14:43,626 - INFO - [PDF] PyMuPDF - Score: 0.97
2025-06-20 14:14:46,048 - INFO - [PDF] pdfplumber - Score: 0.98
2025-06-20 14:14:46,050 - INFO - [PDF] Documento salvo: documentos_processados/Regulamento_Aprovado_2019.md
2025-06-20 14:14:46,051 - INFO - Documento carregado: Regulamento_Aprovado_2019.md
2025-06-20 14:14:46,051 - INFO - Carregando modelo: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
2025-06-20 14:14:46,056 - INFO - Use pytorch device_name: cpu
2025-06-20 14:14:46,056 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Batches: 100%|████████████████████████████████████| 1/1 [00:00<00:00,  6.17it/s]
2025-06-20 14:15:49,425 - INFO - Use pytorch device_name: cpu
2025-06-20 14:15:49,426 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
Batches: 100%|███████████

In [2]:
# 5. Modelo de embedding mais eficiente
MODEL_EMBEDDING = 'sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2'
OLLAMA_API_URL = "http://10.20.50.50:11434/api/generate"
MODEL_LLM = "deepseek-r1:14b"

# 6. Cache de embeddings
try:
    embedding_model = SentenceTransformer(MODEL_EMBEDDING)
    embeddings_documentos = embedding_model.encode(documentos)
except Exception as e:
    logger.error(f"Falha no modelo de embedding: {str(e)}")
    raise

pergunta = "quais são os regulamentos do Dainf?"
try:
    embedding_pergunta = embedding_model.encode([pergunta])[0]
    similaridades = cosine_similarity([embedding_pergunta], embeddings_documentos)[0]
except Exception as e:
    logger.error(f"Erro no cálculo de similaridade: {str(e)}")
    raise

# 7. Análise de resultados aprimorada
logger.info("\nSimilaridade dos documentos:")
for i, (caminho, sim) in enumerate(zip(PDF_PATHS, similaridades)):
    status = "✅" if sim > 0.25 else "❌"
    print(f"{status} Documento {i+1} ({caminho}): {sim:.4f}")

# 8. Threshold dinâmico e seleção múltipla
THRESHOLD = 0.25
contexto = ""
for i, sim in enumerate(similaridades):
    if sim > THRESHOLD:
        logger.info(f"Adicionando documento {i+1} ao contexto (similaridade: {sim:.4f})")
        contexto += f"\n--- DOCUMENTO {i+1} ({PDF_PATHS[i]}) ---\n"
        contexto += documentos[i][:10000]  # Limita o tamanho do contexto
        
if not contexto:
    raise SystemExit("Nenhum documento relevante para a consulta.")

# 9. Prompt otimizado e truncagem segura
MAX_CONTEXT_SIZE = 12000  # Tokens aproximados
if len(contexto) > MAX_CONTEXT_SIZE:
    logger.warning(f"Contexto truncado ({len(contexto)} > {MAX_CONTEXT_SIZE} caracteres)")
    contexto = contexto[:MAX_CONTEXT_SIZE]

# 10. Formatação de prompt aprimorada
prompt = f"""<|im_start|>system
Você é um assistente acadêmico que responde perguntas baseado EXCLUSIVAMENTE nos documentos fornecidos. 
Se a informação não estiver contida nos documentos, responda claramente que não possui os dados necessários.<|im_end|>
<|im_start|>context
{contexto}
<|im_end|>
<|im_start|>user
Pergunta: {pergunta}<|im_end|>
<|im_start|>assistant
Resposta:"""

# 11. Configuração de requisição robusta
payload = {
    "model": MODEL_LLM,
    "prompt": prompt,
    "stream": False,
    "options": {
        "temperature": 0.3,
        "top_p": 0.9,
        "num_ctx": 4096  # Tamanho do contexto
    }
}

try:
    logger.info("Consultando modelo LLM...")
    response = requests.post(
        OLLAMA_API_URL,
        json=payload,
        headers={"Content-Type": "application/json"},
        timeout=120  # 2 minutos timeout
    )
    response.raise_for_status()
    
    resposta = response.json().get('response', '').strip()
    # 12. Limpeza da resposta
    resposta = resposta.split("<|im_end|>")[0].strip()
    resposta = resposta.replace("Resposta:", "").strip()
    
    print("\n" + "="*50)
    print(f"PERGUNTA: {pergunta}")
    print("="*50)
    print(resposta)
    print("="*50)
    
except requests.exceptions.RequestException as e:
    logger.error(f"Erro na requisição: {str(e)}")
except Exception as e:
    logger.error(f"Erro inesperado: {str(e)}")

2025-06-20 13:54:12,782 - INFO - Use pytorch device_name: cpu
2025-06-20 13:54:12,784 - INFO - Load pretrained SentenceTransformer: sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
2025-06-20 13:54:15,155 - ERROR - Falha no modelo de embedding: name 'documentos' is not defined


NameError: name 'documentos' is not defined

In [18]:
similaridades

array([0.43033326, 0.4853886 , 0.30803415], dtype=float32)

In [43]:
# Exportar como Markdown (texto estruturado com títulos e listas)
markdown = resultado.document.export_to_markdown()
print(markdown)

# Exportar como dicionário (JSON)
json_data = resultado.document.export_to_dict()




<!-- image -->

Boletim de Serviço Eletrônico em 16/08/2024

Ministério da Educação UNIVERSIDADE TECNOLÓGICA FEDERAL DO PARANÁ DIRETORIA DE PESQUISA E PÓS-GRADUAÇÃO - CAMPUS CURITIBA COORD. PROG.POS-GRAD. EM TECNOLOGIA - CT

<!-- image -->

## EDITAL Nº 03/2024

## EDITAL DE SELEÇÃO PARA INGRESSO EM 2025 NO CURSO DE MESTRADO EM TECNOLOGIA E SOCIEDADE

- A Diretora Geral do Câmpus Curitiba da Universidade Tecnológica Federal do Paraná (UTFPR), no uso de suas atribuições, torna público o edital do Processo de Seleção para ingresso no Curso de Mestrado do Programa de Pós-Graduação em Tecnologia e Sociedade em 2025, com área de concentração em Tecnologia e Sociedade, conforme processo definido pelo Colegiado do Programa de Pós-Graduação em Tecnologia e Sociedade (PPGTE), e disposto no Regimento Interno do PPGTE.

## 1. DO NÚMERO DE VAGAS

- 1.1. Este processo de seleção visa ao preenchimento de até trinta e seis (36) vagas para o curso de mestrado em Tecnologia e Sociedade.
- 1.2.  De  aco

AttributeError: 'DoclingDocument' object has no attribute 'get_tables'